In [ ]:
# 1. connect to open ai and define embedding model
# 2. prepare vector db(chroma cloud)
# 3. query vector db(chroma cloud)
# 4. pass question and top 2 answers to my llm to restructure answer and make it concise

In [32]:
from langchain_chroma import Chroma
import os
# from pyprojroot import here
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from openai import OpenAI
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()
import chromadb

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_API_KEY")
# os.environ["EMBEDDING_MODEL"] = ""
os.environ["CHROMA_API_KEY"] = os.getenv("CHROMA_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
  
client = chromadb.CloudClient(
  api_key= "ck-8ugJ5sKKBC1ky6L87jWYDbhPTDapdfWPTjTr5tA8fB2h",
  tenant='82905ccd-fe03-4dff-8d77-daab7e235d09',
  database='Test'
)

In [33]:
sql_agent_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

### Prepare files for db

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from pyprojroot import here
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:

# if not os.path.exists(here("Notebooks\\vectorDocs")):
#     os.make

# i dont have to create directory because i am adding to cloud
file_list = os.listdir(here("Notebooks\\vectorDocs"))
docs = [PyPDFLoader(here("Notebooks\\vectorDocs\\KRI GUIDE.pdf")).load_and_split()]

In [16]:
docs = [item for doc in docs for item in doc]

In [17]:
docs

[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-08-14T12:02:07+00:00', 'author': 'user', 'moddate': '2025-08-14T12:02:07+00:00', 'source': 'c:\\Users\\user\\Documents\\NPL\\LLM-WOKG\\Notebooks\\vectorDocs\\KRI GUIDE.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='GUIDE FOR FUTURE KRI DOCUMENTS \n1. Unit – No comments  \n2. SN – No comments \n3. KRI Standard – No comments \n4. Key Risk Indicator \na. As part of ETL findings, some identified risks are not potential risks the company \nfaces. The document must be reevaluated for data quality purposes. For \nexample, ‘# total number of requests received’ (Total media correspondences \n(including social media) that have been reported and percentage of closed \nrequests) is not a risk to the company. \n5. Basis for compilation of KRI data – No comments \n6. Risk Category  \na. For dashboarding risk should be categorized by the different types of risks, not 

In [15]:
type(docs)

list

In [18]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)

In [20]:
doc_splits = text_splitter.split_documents(docs)

In [21]:
doc_splits

[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-08-14T12:02:07+00:00', 'author': 'user', 'moddate': '2025-08-14T12:02:07+00:00', 'source': 'c:\\Users\\user\\Documents\\NPL\\LLM-WOKG\\Notebooks\\vectorDocs\\KRI GUIDE.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='GUIDE FOR FUTURE KRI DOCUMENTS \n1. Unit – No comments  \n2. SN – No comments \n3. KRI Standard – No comments \n4. Key Risk Indicator \na. As part of ETL findings, some identified risks are not potential risks the company \nfaces. The document must be reevaluated for data quality purposes. For \nexample, ‘# total number of requests received’ (Total media correspondences \n(including social media) that have been reported and percentage of closed \nrequests) is not a risk to the company. \n5. Basis for compilation of KRI data – No comments \n6. Risk Category  \na. For dashboarding risk should be categorized by the different types of risks, not 

In [23]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(doc_splits))]

In [ ]:
# vectordb = Chroma.from_documents(
#                 documents=doc_splits,
#                 collection_name=self.collection_name,
#                 embedding=OpenAIEmbeddings(model=self.embedding_model),
#                 persist_directory=str(here(self.vectordb_dir))
#             )

In [22]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
    tenant="82905ccd-fe03-4dff-8d77-daab7e235d09",
    database="Test",
)

In [24]:
vector_store.add_documents(documents=doc_splits, ids=uuids)

['6b3088f5-8fd3-4ed4-8aa5-a893a5b8d67b',
 'd59f4235-3376-41e6-b862-518a85a46f24',
 '05728f6d-cc86-4150-ae7c-deafde3310e9',
 '98b3c0b6-f5dc-4975-a929-4c996fb1242a',
 'fd43e972-47ae-4e71-b315-713f8b76137d']

In [25]:
print("Number of vectors in vectordb:",
      vector_store._collection.count(), "\n\n")

Number of vectors in vectordb: 5 




In [34]:
message = "how do i represent currencies in my kri document?"

In [36]:
vectorAns = vector_store.similarity_search(message, k=2)

In [37]:
vectorAns

[Document(id='98b3c0b6-f5dc-4975-a929-4c996fb1242a', metadata={'author': 'user', 'creationdate': '2025-08-14T12:02:07+00:00', 'creator': 'Microsoft® Word LTSC', 'producer': 'Microsoft® Word LTSC', 'source': 'c:\\Users\\user\\Documents\\NPL\\LLM-WOKG\\Notebooks\\vectorDocs\\KRI GUIDE.pdf', 'moddate': '2025-08-14T12:02:07+00:00', 'page_label': '2', 'total_pages': 3, 'page': 1}, page_content='characters. Do not do Ghanaian Cedi or Gh.Cedi, instead do GHC. Also, separate \ncurrency and value with a space. \nl. All comparisons should be signs and not be in natural language. Instead of Shall \nnot be lower than or Shall not exceed, do >= and <= respectively. \nm. Comparisons should not have spaces e.g. > = should be >=. \nn. All threshold limits should be expressed as numbers and not in natural language. \nValues like Established & operational, Full policy coverage, and Licensed(0) \nshould not be in the column. Numeric equivalents should be used instead. \n8. Tolerance Limit  \na. All instr

In [38]:
retrieved_content = [ans.page_content for ans in vectorAns]
content = "\n\n".join(retrieved_content)
content

'characters. Do not do Ghanaian Cedi or Gh.Cedi, instead do GHC. Also, separate \ncurrency and value with a space. \nl. All comparisons should be signs and not be in natural language. Instead of Shall \nnot be lower than or Shall not exceed, do >= and <= respectively. \nm. Comparisons should not have spaces e.g. > = should be >=. \nn. All threshold limits should be expressed as numbers and not in natural language. \nValues like Established & operational, Full policy coverage, and Licensed(0) \nshould not be in the column. Numeric equivalents should be used instead. \n8. Tolerance Limit  \na. All instruction for 6. Threshold Limits apply to Tolerance Limit. \nb. If tolerance limit is a range, eg. >0, <=1 upper bound of the range should be used. \nInstead of >0, <=1 do <=1. Note that upper bound may differ according to KRI \nmeasurement. E.g. <100%, >=60% should be >=60%.\n\nGUIDE FOR FUTURE KRI DOCUMENTS \n1. Unit – No comments  \n2. SN – No comments \n3. KRI Standard – No comments \n4.

In [30]:
from langchain_core.prompts import ChatPromptTemplate

In [39]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        '''
        You are an assistant that answers questions about a guide for preparing a key risk indicator document for a company. Given a user question {question} and some documents {documents}, provides concise answer by looking through the documents.
        '''
    ),
    ("human",
     '''
    User question: {question}
    Documents: {documents}

    Provide a concise answer
'''
     )
])

In [41]:
rag = prompt.invoke({
    "question": message, 
    "documents": retrieved_content
})
response = sql_agent_llm.invoke(rag)

In [43]:
response.content

'To represent currencies in your KRI document, use GHC instead of Ghanaian Cedi or Gh.Cedi. Also, separate currency and value with a space.'